In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from terrain import *
from functions import *

In [150]:
class Climber():
    
    def __init__(self,position_index,terrain):
        self.n_input = 20
        self.n_hidden = 5
        self.n_output = 1
        self.learning_rate = 1
        
        #Outside world
        self.terrain = terrain
        self.position_index = position_index
        
        
        # inputs and expected outputs
        self.old_input = self.terrain[0,self.position_index:self.position_index + self.n_input].reshape(self.n_input,1)
        self.input = self.terrain[0,self.position_index:self.position_index + self.n_input].reshape(self.n_input,1)
        self.output = None #Shape 1,1
        self.y = None
        
        #Weights
        self.W1 = None
        self.b1 = None
        self.W2 = None
        self.b2 = None
        
        self.costs = []
        self.mistakes = 0
        self.position_switch_counter = 0

    def initialize_weights(self):
        self.W1 = np.random.normal(0,1/np.sqrt(self.n_input),size=[self.n_hidden,self.n_input])#Shape 5,10
        self.W2 = np.random.normal(0,1/np.sqrt(self.n_hidden),size=[self.n_output,self.n_hidden])#Shape 1,5
        self.b1 = np.zeros((self.n_hidden,1))#Shape 5,1
        self.b2 = np.zeros((self.n_output,1))#shape 1,1
        
    def feed_forward(self):
        standardized_input =self.standardize(self.input)# self.input.copy()# BOOM!!!!
        z1 = np.dot(self.W1,standardized_input)+self.b1  #Shape 5,1
        a1 = sigmoid(z1)  #Shape 5,1
        z2 = np.dot(self.W2,a1)+self.b2  #Shape 1,1
        a2 = sigmoid(z2)   #Shape 1,1
        self.output = a2.copy()
        
        
        values = {"standardized_input":standardized_input.copy(),"z1":z1,"a1":a1,"z2":z2,"a2":a2}
        return values
    
    def calculate_cost(self):
        
        values = self.feed_forward()
        self.move()
        diff = np.mean(self.input) - np.mean(self.old_input)
        #print("Difference is :"+ str(diff))
        if np.absolute(diff)>0:
        #print("Mean of terrain is :",np.mean(terrain))
            #print(diff)
            self.y = self.cost_helper(diff,self.output) ##IMPORTANT

            c = (-np.dot(self.y,np.log(self.output).T)-np.dot(1-self.y,np.log(1-self.output).T))/1
            #print("Cost is :",c)
            self.costs.append(c)

            gradients = self.back_popagate(values)
            self.gradient_descent(gradients)
        
        
    def back_popagate(self,values):
        m = 1
        dz2 = values["a2"]-self.y  #SHAPE 1,1
        dW2 = np.dot(dz2,values["a1"].T)/m #SHAPE 1,5
        db2 = np.sum(dz2,axis=1,keepdims=True)/m

        dz1 = np.multiply(np.dot(self.W2.T,dz2),sigmoid_prime(values["z1"]))  #SHAPE 5,1
        dW1 = np.dot(dz1,values["standardized_input"].T)/m #SHAPE 5,10
        db1 = np.sum(dz1,axis=1,keepdims=True)/m
        
        gradients = {"dW2":dW2,"db2":db2,"dW1":dW1,"db1":db1}
        return gradients
        
     
    def gradient_descent(self,gradients):
        self.W2 = self.W2 - self.learning_rate*gradients["dW2"]
        self.W1 = self.W1 - self.learning_rate*gradients["dW1"]
        self.b2 = self.b2 - self.learning_rate*gradients["db2"]
        self.b1 = self.b1 -self.learning_rate*gradients["db1"]
        
    def move(self):
        X = None
        if self.output.reshape(1,) > 0.5:
            if self.position_index < terrain.shape[1] - (self.n_input+3):
                #Move Right
                self.position_index = self.position_index + 3
                X = terrain[0,self.position_index:self.position_index + self.n_input]
                
            else :
                print("Right edge reached")
                self.position_index = np.random.randint(450,480)
                self.position_switch_counter = self.position_switch_counter + 1
                
    
                
        else :
            if self.position_index<3:
                print("Left edge reached")
                self.position_index = np.random.randint(510,550)
                self.position_switch_counter = self.position_switch_counter + 1
                
            
                
            else :
                #Move Left
                self.position_index = self.position_index - 3
                X = terrain[0,self.position_index:self.position_index + self.n_input]
        
        if X is not None:
            self.old_input = self.input
            self.input = X.reshape(self.n_input,1)
        else:
            self.old_input = self.terrain[0,self.position_index:self.position_index + self.n_input].reshape(self.n_input,1)
            self.input = self.terrain[0,self.position_index:self.position_index + self.n_input].reshape(self.n_input,1)
            self.output = None
            self.y = 0
            
            
    #Write to calculate the number of wrong decision made
    def cost_helper(self,diff,number):
        if diff > 0:
            if number>0.5:
                y = 1
            else :
                y = 0
        else:
            self.mistakes = self.mistakes + 1
            if number >0.5:
                y = 0
            else :
                y = 1
        return y
    
    def change_terrain(self,terrain):
        self.terrain = terrain
        
        
    def standardize(self,z):
        return (z-np.mean(z))/np.std(z)
        

In [151]:
#SCRIPTING

In [152]:
terrain = get_parabola(1000)
climber = Climber(349,terrain)
climber.initialize_weights()

In [166]:
%matplotlib notebook
#f = plt.figure()
#plot = f.add_subplot(111)
#plt.ion()
y = []
for i in range(5000):
    
 
    print("Output after iteration "+str(i)+" is  : "+str(climber.output))
    climber.calculate_cost()
    
    
    
    #plot.clear()
    #plot.scatter(range(1000),terrain)
    #plot.scatter(climber.position_index,terrain[0,climber.position_index],c="r",marker="o")
    #plot.set_ylabel("Output :"+str(climber.output))
    #f.tight_layout()
    #f.canvas.draw()
    
    #When Climber gets stuck
    y.append(terrain[0,climber.position_index])
    if(len(y)>10):
        if(np.absolute(np.mean(y[-9:])-y[-1])<0.05 ):
            
            #print("-------NEW POSITION-------")
            climber.position_index =np.random.randint(450,550)#i%975 + 3 #
            climber.old_input = terrain[0,climber.position_index:climber.position_index + climber.n_input].reshape(climber.n_input,1)
            climber.input = terrain[0,climber.position_index:climber.position_index + climber.n_input].reshape(climber.n_input,1)
            climber.output = None
            climber.y = 0
            
            climber.position_switch_counter = climber.position_switch_counter + 1
            y = []
    
    
    
    #if(terrain[0,climber.position_index]>0.95):
        #climber.position_index = np.random.randint(3,986)
        #print("-------NEW POSITION-------")
    

Output after iteration 0 is  : [[  4.54803288e-05]]
Output after iteration 1 is  : [[  4.54732459e-05]]
Output after iteration 2 is  : [[  4.54662139e-05]]
Output after iteration 3 is  : [[  4.54592289e-05]]
Output after iteration 4 is  : [[  4.54522872e-05]]
Output after iteration 5 is  : [[  4.54453857e-05]]
Output after iteration 6 is  : [[  4.54385214e-05]]
Output after iteration 7 is  : [[  4.54316917e-05]]
Output after iteration 8 is  : [[  4.54248944e-05]]
Output after iteration 9 is  : [[  4.54181271e-05]]
Output after iteration 10 is  : [[  4.54113880e-05]]
Output after iteration 11 is  : [[  4.54046753e-05]]
Output after iteration 12 is  : [[  4.53979873e-05]]
Output after iteration 13 is  : [[  4.53913227e-05]]
Output after iteration 14 is  : [[  4.53846799e-05]]
Output after iteration 15 is  : [[  4.53780578e-05]]
Output after iteration 16 is  : [[  4.53714552e-05]]
Output after iteration 17 is  : [[  4.53648711e-05]]
Output after iteration 18 is  : [[  4.53583044e-05]]
Out

Output after iteration 226 is  : [[ 0.99996301]]
Output after iteration 227 is  : [[ 0.99996301]]
Output after iteration 228 is  : [[ 0.99996302]]
Output after iteration 229 is  : [[ 0.99996302]]
Output after iteration 230 is  : [[ 0.99996303]]
Output after iteration 231 is  : [[ 0.99996304]]
Output after iteration 232 is  : [[ 0.99996304]]
Output after iteration 233 is  : [[ 0.99996305]]
Output after iteration 234 is  : [[ 0.99996305]]
Output after iteration 235 is  : [[ 0.99996306]]
Output after iteration 236 is  : [[ 0.99996306]]
Output after iteration 237 is  : [[ 0.99996307]]
Output after iteration 238 is  : [[ 0.99996307]]
Output after iteration 239 is  : [[ 0.99996308]]
Output after iteration 240 is  : [[ 0.99996308]]
Output after iteration 241 is  : [[ 0.99996309]]
Output after iteration 242 is  : [[ 0.9999631]]
Output after iteration 243 is  : [[ 0.9999631]]
Output after iteration 244 is  : [[ 0.99996311]]
Output after iteration 245 is  : [[ 0.99996311]]
Output after iteration

Output after iteration 417 is  : [[  4.41250739e-05]]
Output after iteration 418 is  : [[  4.41191766e-05]]
Output after iteration 419 is  : [[  4.41132815e-05]]
Output after iteration 420 is  : [[  4.41073887e-05]]
Output after iteration 421 is  : [[  4.41014981e-05]]
Output after iteration 422 is  : [[  4.40956097e-05]]
Output after iteration 423 is  : [[  4.40897234e-05]]
Output after iteration 424 is  : [[  4.40838394e-05]]
Output after iteration 425 is  : [[  4.40779575e-05]]
Output after iteration 426 is  : [[  4.40720777e-05]]
Output after iteration 427 is  : [[  4.40662001e-05]]
Output after iteration 428 is  : [[  4.40603246e-05]]
Output after iteration 429 is  : [[  4.40544512e-05]]
Output after iteration 430 is  : [[  4.40485798e-05]]
Output after iteration 431 is  : [[  4.40427106e-05]]
Output after iteration 432 is  : [[  4.40368435e-05]]
Output after iteration 433 is  : [[  4.40309784e-05]]
Output after iteration 434 is  : [[  4.40251153e-05]]
Left edge reached
Output aft

Output after iteration 634 is  : [[  4.40155964e-05]]
Output after iteration 635 is  : [[  4.40093503e-05]]
Output after iteration 636 is  : [[  4.40031234e-05]]
Output after iteration 637 is  : [[  4.39969145e-05]]
Output after iteration 638 is  : [[  4.39907228e-05]]
Output after iteration 639 is  : [[  4.39845473e-05]]
Output after iteration 640 is  : [[  4.39783872e-05]]
Output after iteration 641 is  : [[  4.39722416e-05]]
Output after iteration 642 is  : [[  4.39661100e-05]]
Output after iteration 643 is  : [[  4.39599917e-05]]
Output after iteration 644 is  : [[  4.39538859e-05]]
Output after iteration 645 is  : [[  4.39477923e-05]]
Output after iteration 646 is  : [[  4.39417101e-05]]
Output after iteration 647 is  : [[  4.39356390e-05]]
Output after iteration 648 is  : [[  4.39295784e-05]]
Output after iteration 649 is  : [[  4.39235279e-05]]
Output after iteration 650 is  : [[  4.39174871e-05]]
Output after iteration 651 is  : [[  4.39114556e-05]]
Output after iteration 652 i

Output after iteration 861 is  : [[ 0.9999642]]
Output after iteration 862 is  : [[ 0.9999642]]
Output after iteration 863 is  : [[ 0.99996421]]
Output after iteration 864 is  : [[ 0.99996421]]
Output after iteration 865 is  : [[ 0.99996422]]
Output after iteration 866 is  : [[ 0.99996422]]
Output after iteration 867 is  : [[ 0.99996423]]
Output after iteration 868 is  : [[ 0.99996423]]
Output after iteration 869 is  : [[ 0.99996424]]
Output after iteration 870 is  : [[ 0.99996424]]
Output after iteration 871 is  : [[ 0.99996425]]
Output after iteration 872 is  : [[ 0.99996425]]
Output after iteration 873 is  : [[ 0.99996426]]
Output after iteration 874 is  : [[ 0.99996426]]
Output after iteration 875 is  : [[ 0.99996427]]
Output after iteration 876 is  : [[ 0.99996427]]
Output after iteration 877 is  : [[ 0.99996428]]
Output after iteration 878 is  : [[ 0.99996428]]
Output after iteration 879 is  : [[ 0.99996429]]
Output after iteration 880 is  : [[ 0.99996429]]
Output after iteration

...

Output after iteration 3997 is  : [[  3.77284153e-05]]
Output after iteration 3998 is  : [[  3.77239021e-05]]
Output after iteration 3999 is  : [[  3.77193978e-05]]
Output after iteration 4000 is  : [[  3.77149022e-05]]
Output after iteration 4001 is  : [[  3.77104148e-05]]
Output after iteration 4002 is  : [[  3.77059354e-05]]
Output after iteration 4003 is  : [[  3.77014634e-05]]
Output after iteration 4004 is  : [[  3.76969987e-05]]
Output after iteration 4005 is  : [[  3.76925410e-05]]
Output after iteration 4006 is  : [[  3.76880900e-05]]
Output after iteration 4007 is  : [[  3.76836455e-05]]
Output after iteration 4008 is  : [[  3.76792071e-05]]
Output after iteration 4009 is  : [[  3.76747747e-05]]
Output after iteration 4010 is  : [[  3.76703481e-05]]
Output after iteration 4011 is  : [[  3.76659270e-05]]
Output after iteration 4012 is  : [[  3.76615114e-05]]
Output after iteration 4013 is  : [[  3.76571009e-05]]
Output after iteration 4014 is  : [[  3.76526955e-05]]
Output aft

Output after iteration 4223 is  : [[ 0.99996917]]
Output after iteration 4224 is  : [[ 0.99996917]]
Output after iteration 4225 is  : [[ 0.99996917]]
Output after iteration 4226 is  : [[ 0.99996918]]
Output after iteration 4227 is  : [[ 0.99996918]]
Output after iteration 4228 is  : [[ 0.99996918]]
Output after iteration 4229 is  : [[ 0.99996919]]
Output after iteration 4230 is  : [[ 0.99996919]]
Output after iteration 4231 is  : [[ 0.9999692]]
Output after iteration 4232 is  : [[ 0.9999692]]
Output after iteration 4233 is  : [[ 0.9999692]]
Output after iteration 4234 is  : [[ 0.99996921]]
Output after iteration 4235 is  : [[ 0.99996921]]
Output after iteration 4236 is  : [[ 0.99996921]]
Output after iteration 4237 is  : [[ 0.99996922]]
Output after iteration 4238 is  : [[ 0.99996922]]
Output after iteration 4239 is  : [[ 0.99996923]]
Output after iteration 4240 is  : [[ 0.99996923]]
Output after iteration 4241 is  : [[ 0.99996923]]
Output after iteration 4242 is  : [[ 0.99996924]]
Out

Output after iteration 4456 is  : [[ 0.99996928]]
Output after iteration 4457 is  : [[ 0.99996928]]
Output after iteration 4458 is  : [[ 0.99996928]]
Output after iteration 4459 is  : [[ 0.99996929]]
Output after iteration 4460 is  : [[ 0.99996929]]
Output after iteration 4461 is  : [[ 0.99996929]]
Output after iteration 4462 is  : [[ 0.9999693]]
Output after iteration 4463 is  : [[ 0.9999693]]
Output after iteration 4464 is  : [[ 0.9999693]]
Output after iteration 4465 is  : [[ 0.99996931]]
Output after iteration 4466 is  : [[ 0.99996931]]
Output after iteration 4467 is  : [[ 0.99996932]]
Output after iteration 4468 is  : [[ 0.99996932]]
Output after iteration 4469 is  : [[ 0.99996932]]
Output after iteration 4470 is  : [[ 0.99996933]]
Output after iteration 4471 is  : [[ 0.99996933]]
Output after iteration 4472 is  : [[ 0.99996933]]
Output after iteration 4473 is  : [[ 0.99996934]]
Output after iteration 4474 is  : [[ 0.99996934]]
Output after iteration 4475 is  : [[ 0.99996935]]
Out

Output after iteration 4680 is  : [[  3.64499676e-05]]
Output after iteration 4681 is  : [[  3.64459335e-05]]
Output after iteration 4682 is  : [[  3.64419009e-05]]
Output after iteration 4683 is  : [[  3.64378698e-05]]
Output after iteration 4684 is  : [[  3.64338403e-05]]
Output after iteration 4685 is  : [[  3.64298122e-05]]
Output after iteration 4686 is  : [[  3.64257855e-05]]
Output after iteration 4687 is  : [[  3.64217603e-05]]
Output after iteration 4688 is  : [[  3.64177366e-05]]
Output after iteration 4689 is  : [[  3.64137143e-05]]
Output after iteration 4690 is  : [[  3.64096933e-05]]
Output after iteration 4691 is  : [[  3.64056738e-05]]
Output after iteration 4692 is  : [[  3.64016557e-05]]
Output after iteration 4693 is  : [[  3.63976390e-05]]
Output after iteration 4694 is  : [[  3.63936236e-05]]
Output after iteration 4695 is  : [[  3.63896096e-05]]
Output after iteration 4696 is  : [[  3.63855969e-05]]
Output after iteration 4697 is  : [[  3.63815856e-05]]
Output aft

Output after iteration 4904 is  : [[  3.63542463e-05]]
Output after iteration 4905 is  : [[  3.63499441e-05]]
Output after iteration 4906 is  : [[  3.63456563e-05]]
Output after iteration 4907 is  : [[  3.63413819e-05]]
Output after iteration 4908 is  : [[  3.63371204e-05]]
Output after iteration 4909 is  : [[  3.63328709e-05]]
Output after iteration 4910 is  : [[  3.63286329e-05]]
Output after iteration 4911 is  : [[  3.63244059e-05]]
Output after iteration 4912 is  : [[  3.63201892e-05]]
Output after iteration 4913 is  : [[  3.63159824e-05]]
Output after iteration 4914 is  : [[  3.63117850e-05]]
Output after iteration 4915 is  : [[  3.63075965e-05]]
Output after iteration 4916 is  : [[  3.63034166e-05]]
Output after iteration 4917 is  : [[  3.62992450e-05]]
Output after iteration 4918 is  : [[  3.62950811e-05]]
Output after iteration 4919 is  : [[  3.62909247e-05]]
Output after iteration 4920 is  : [[  3.62867756e-05]]
Output after iteration 4921 is  : [[  3.62826333e-05]]
Output aft

In [167]:
climber.position_switch_counter

162

In [168]:
climber.mistakes

2

In [156]:
print(climber.mistakes/climber.position_switch_counter)

0.0625


In [50]:
climber.mistakes = 0

In [51]:
climber.position_switch_counter = 0